In [254]:
import cobra

In [255]:
met = cobra.io.read_sbml_model("./data/iMcBath_rewritten.xml")

In [257]:
met.optimize()

,fluxes,reduced_costs
2OH3K5MPPISO,0.000000,0.000000e+00
34DHOXPEGOX,0.000000,-0.000000e+00
3HAD100,-0.005615,-0.000000e+00
3HAD120,0.005615,0.000000e+00
3HAD140,0.003835,0.000000e+00
...,...,...
hemo_RNA,0.000000,2.775558e-16
hemo_protein,0.000000,-2.843376e+00
Hemo_Biosynthesis,0.000000,0.000000e+00
Take_Out_Hemo,0.000000,0.000000e+00


In [258]:
met.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.01244,0,0.00%
ch4_e,EX_ch4_e,18.46,1,100.00%
cl_e,EX_cl_e,0.03817,0,0.00%
cobalt2_e,EX_cobalt2_e,1.027E-05,0,0.00%
cu2_e,EX_cu2_e,0.000269,0,0.00%
fe2_e,EX_fe2_e,0.00109,0,0.00%
h_e,EX_h_e,1.458,0,0.00%
k_e,EX_k_e,0.03142,0,0.00%
mg2_e,EX_mg2_e,0.02198,0,0.00%
mn2_e,EX_mn2_e,3.205E-06,0,0.00%


In [231]:
met.reactions.EX_co2_e.reaction

'co2_e --> '

In [205]:
met.metabolites.co2_e

Metabolite identifier,co2_e
Name,CO(2)
Memory address,0x07fd088b80130
Formula,CO2
Compartment,e
In 2 reaction(s),"EX_co2_e, CO2t"


It is clearly seen that in M. capsulatus the uptake of methane outstands among all the other uptakes. Thus, now we will check those reactions where methane is involved. Besides, we will also check the methanol-involved reactions as it is here where our main interest is placed.

IMPORTANT!! Consider using the query both with initial letter as capital letter and not capital!

In [45]:
met.reactions.query("growth", "name")

[]

In [46]:
met.reactions.ACPS1

Reaction identifier,ACPS1
Name,Acyl-carrier protein synthase
Memory address,0x07fd08adedfa0
Stoichiometry,"apoACP_c + coa_c --> ACP_c + h_c + pap_c apo-[acyl-carrier protein] + CoA --> holo-[acyl-carrier protein] + H(+) + adenosine 3',5'-bisphosphate"
GPR,MCA1522
Lower bound,0.0
Upper bound,0.0


In [47]:
met.metabolites.cys__L_c

Metabolite identifier,cys__L_c
Name,L-cysteine zwitterion
Memory address,0x07fd0cbb42880
Formula,C3H7NO2S
Compartment,c
In 12 reaction(s),"GLUCYS, PPNCL2, PPNCL3, CYSTRS, THZPSN, CYSS_trdrd, CYSS, CYSDS, SK_cys__L_c, CYSTA, BIOMASS_Mcapsulatus, AMPTASECG"


In [28]:
met.groups()

TypeError: 'DictList' object is not callable

Question: Why is it that the lb is placed to -18.46? Also interesting that this is exactly the optimal value obtained when optimizing.

Obtained from the paper for the model:
The uptake rate for methane in iMcBath was constrained to 18.46 mmol gDW-1 h-1, a value adopted from de la Torre et al. (2015) due to the lack of a specific measurement for M. capsulatus (Bath).

Should we take this limit value from the experimental killing curves from wet lab?

In [9]:
met.reactions.CH4tex

Reaction identifier,CH4tex
Name,methane transport via diffusion (extracellular to periplasm)
Memory address,0x07f69d16f2d30
Stoichiometry,ch4_e <=> ch4_p methane <=> methane
GPR,
Lower bound,-1000.0
Upper bound,1000.0


Question: Why do we consider the diffusion reaction without any limit? What is the relation between the methane obtained from the exchange reaction level and the diffusion? Because if we consider the diffusion with no limit and the is this periplasmatic methane that is taking further down on the reactions, where does the 18.46 limit apply?

In [27]:
print(met.reactions.SMMOi)
print(met.reactions.PMMOipp)
print(met.reactions.PMMODCipp)

SMMOi: ch4_c + h_c + nadh_c + o2_c --> h2o_c + meoh_c + nad_c
PMMOipp: ch4_p + o2_p + q8h2_im --> h2o_p + meoh_p + q8_im
PMMODCipp: ch4_p + 2.0 focytcc555_p + 2.0 h_p + o2_p --> 2.0 ficytcc555_p + h2o_p + meoh_p


For our interest in K. Phaffi:

In [173]:
# bounds for this pMMO reaction are at 0 already

met.reactions.PMMOipp

Reaction identifier,PMMOipp
Name,particulate Methane Monooxygenase
Memory address,0x07fd08a5acd00
Stoichiometry,ch4_p + o2_p + q8h2_im --> h2o_p + meoh_p + q8_im methane + dioxygen + Ubiquinol-8 --> H2O + methanol + Ubiquinone-8
GPR,( ( MCA1796 and MCA1797 and MCA1798 ) or ( MCA2853 and MCA2854 and MCA2855 ) ) and MCA0295
Lower bound,0
Upper bound,0


In [169]:
# bounds for this pMMO reaction are at 0 already


met.reactions.PMMODCipp

Question: Difference between both? Any reason why both are constrained not to work?

Now we come to the methanol reactions:

In [16]:
met.reactions.EX_meoh_e

Reaction identifier,EX_meoh_e
Name,methanol exchange
Memory address,0x0185c3e44978
Stoichiometry,meoh_e --> methanol -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [6]:
met.reactions.MEOHtex

Reaction identifier,MEOHtex
Name,methanol transport via diffusion (extracellular to periplasm)
Memory address,0x010d90ab9ef0
Stoichiometry,meoh_e <=> meoh_p methanol <=> methanol
GPR,
Lower bound,-1000.0
Upper bound,1000.0


This following reaction is the one that we might need to introduce to out K. phaffii model as it is the one getting from methanol to formaldehyde, starting the MUT pathway?

In [17]:
met.reactions.MEOHDHipp

Reaction identifier,MEOHDHipp
Name,methanol dehydrogenase (cytochrome c)
Memory address,0x0185c405f588
Stoichiometry,2.0 ficytcc555_p + meoh_p --> fald_p + 2.0 focytcc555_p + 2.0 h_p 2.0 ferricytochrome c + methanol --> formaldehyde + 2.0 ferrocytochrome c + 2.0 H(+)
GPR,( ( MCA0299 and MCA0300 and MCA1528 and MCA1529 and MCA1530 and MCA1525 and MCA1527 ) or (...
Lower bound,0.0
Upper bound,1000.0


In [22]:
met.reactions.MEOHtrpp

Reaction identifier,MEOHtrpp
Name,Methanol reversible transport via diffusion (periplasm)
Memory address,0x010d90ab9cc0
Stoichiometry,meoh_p <=> meoh_c methanol <=> methanol
GPR,
Lower bound,-1000.0
Upper bound,1000.0


Also, easily obtained by:

In [17]:
met.metabolites.ch4_c.summary()

Percent,Flux,Reaction,Definition
100.00%,18.46,CH4trpp,ch4_p <=> ch4_c
Percent,Flux,Reaction,Definition
100.00%,-18.46,SMMOi,ch4_c + h_c + nadh_c + o2_c --> h2o_c + meoh_c + nad_c


Notice that in this model, one can refer to extracellular species by adding e, periplasmatic by adding p and intracellular by adding c (cytosol). And by checking the file im refers to inner membrane. This will depend on the model, tends to be specified somewhere on the sbml file under compartments.

In [16]:
met.metabolites.meoh_c.summary()

Percent,Flux,Reaction,Definition
100.00%,18.46,SMMOi,ch4_c + h_c + nadh_c + o2_c --> h2o_c + meoh_c + nad_c
Percent,Flux,Reaction,Definition
100.00%,-18.46,MEOHtrpp,meoh_p <=> meoh_c


In [62]:
met.medium

{'EX_ca2_e': 1000.0,
 'EX_cd2_e': 1000.0,
 'EX_ch4_e': 18.46,
 'EX_cl_e': 1000.0,
 'EX_co_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_no3_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_zn2_e': 1000.0}

In [20]:
met.reactions.FOMETRi

Reaction identifier,FOMETRi
Name,Aminomethyltransferase
Memory address,0x07f69d1598fd0
Stoichiometry,"h2o_c + methf_c --> 5fthf_c + h_c H2O + (6R)-5,10-methenyltetrahydrofolate --> 5-Formyltetrahydrofolate + H(+)"
GPR,MCA0350
Lower bound,0.0
Upper bound,1000.0


In [143]:
met.reactions.SMMOi

Reaction identifier,SMMOi
Name,soluble Methane Monooxygenase
Memory address,0x07fd08920fa90
Stoichiometry,ch4_c + h_c + nadh_c + o2_c --> h2o_c + meoh_c + nad_c methane + H(+) + NADH(2-) + dioxygen --> H2O + methanol + NAD(+)
GPR,MCA1194 and MCA1195 and MCA1198 and MCA1196 and MCA1200 and MCA1202 and MCA1205
Lower bound,0.0
Upper bound,1000.0


In [117]:
met.metabolites.query("uracil","name")

[<Metabolite 4r5au_c at 0x7fd0cbbd3460>,
 <Metabolite 5aprbu_c at 0x7fd0cbbd3580>,
 <Metabolite 5apru_c at 0x7fd0cbbd35b0>,
 <Metabolite ura_c at 0x7fd08af363a0>]

In [116]:
met.metabolites.csn_c

Metabolite identifier,csn_c
Name,cytosine
Memory address,0x07fd0cbb420a0
Formula,C4H5N3O
Compartment,c
In 1 reaction(s),CSND


In [233]:
met.compartments

{'c': 'cytosol',
 'e': 'extracellular',
 'p': 'periplasm',
 'im': 'inner membrane',
 'C_c': '',
 'C_e': ''}

In [256]:
# now we will introduce the heterologous protein leghemoglobin with reactions for dna replication,
# transcription and translation
# as the sequences are long, we calculate the stoichiometry with a script which is on the github repo and
# based on the logic behind introduction of heterologous protein production in the paper:
# Irani ZA, Kerkhoven EJ, Shojaosadati SA, Nielsen J (2016) Genome-scale metabolic model of Pichia pastoris with native and humanized glycosylation of recombinant proteins. Biotechnol Bioeng 113:961–969. doi: 10.1002/bit.25863.

# make new reactions



hemo_DNA_reaction = cobra.Reaction(
            'hemo_DNA',
            name = 'Leghemoglobin DNA replication',
            lower_bound = 0.0,
            upper_bound = 1000.0
        )

hemo_RNA_reaction = cobra.Reaction(
            'hemo_RNA',
            name = 'Leghemoglobin transcription',
            lower_bound = 0.0,
            upper_bound = 1000.0
        )

hemo_AA_reaction = cobra.Reaction(
            'hemo_protein',
            name = 'Leghemoglobin translation',
            lower_bound = 0.0,
            upper_bound = 1000.0
)


Take_Out_Hemo = cobra.Reaction(
            'Take_Out_Hemo',
            name = 'Take_Out_Hemo',
            lower_bound = -1000.0,
            upper_bound = 1000.0
)



Hemo_Biosynthesis = cobra.Reaction(
            'Hemo_Biosynthesis',
            name = 'Leghemoglobin Biosynthesis',
            lower_bound = 0.0,
            upper_bound = 1000.0
)

EX_hemo = cobra.Reaction(
            'EX_hemo',
            name = 'Leghemoglobin exchange reaction',
            lower_bound = 0.0,
            upper_bound = 1000.0
)



# add the metabolites to be produced by these reactions


Hemo_DNA = cobra.Metabolite(
    'Hemo_DNA',
    formula='',
    name='Hemo_DNA',
    compartment='c')

Hemo_RNA = cobra.Metabolite(
    'Hemo_RNA',
    formula='',
    name='Hemo_RNA',
    compartment='c')

Hemo_AA = cobra.Metabolite(
    'Hemo_AA',
    formula='',
    name='Leghemoglobin',
    compartment='c')

hemo_c = cobra.Metabolite(
    'hemo_c',
    formula='',
    name='hemo_c',
    compartment='c')

hemo_e = cobra.Metabolite(
    'hemo_e',
    formula='',
    name='hemo_e',
    compartment='e')


met.add_metabolites([hemo_c,
                     hemo_e,
                     Hemo_DNA,
                     Hemo_RNA,
                     Hemo_AA])



# add involved metabolites and stoichiometry



hemo_DNA_reaction.add_metabolites(
    {
        ### deoxynucleotides
        # A
        met.metabolites.adn_c: -1.0424,
        # T
        met.metabolites.thym_c: -1.0424,
        # C
        met.metabolites.csn_c: -0.57636,
        # G
        met.metabolites.gua_c: -0.57636,
        # ATP
        met.metabolites.atp_c: -11.00757,
        # H2O
        met.metabolites.h2o_c: -11.00757,
        # Hemo_DNA
        met.metabolites.Hemo_DNA: 1.0,
        # ADP
        met.metabolites.adp_c: 11.00757,
        # Phosphate
        met.metabolites.pi_c: 11.00757
    }
)

hemo_RNA_reaction.add_metabolites(
    {
        ### RNA nucleotides
        # A
        met.metabolites.adn_c: -1.2408,
        # U
        met.metabolites.ura_c: -0.80329,
        # C
        met.metabolites.csn_c: -0.6455,
        # G
        met.metabolites.gua_c: -0.43034,
        # ATP
        met.metabolites.atp_c: -7.48783,
        # H2O
        met.metabolites.h2o_c: -7.48783,
        # Hemo_RNA
        met.metabolites.Hemo_RNA: 1.0,
        # ADP
        met.metabolites.atp_c: 7.48783,
        # Phosphate
        met.metabolites.pi_c: 7.4878
    }
)

hemo_AA_reaction.add_metabolites(
    {
        ### Amino Acids
        # A
        met.metabolites.ala__L_c: -1.48373,
        # C, no Cystein in leghemoglobin
        # pheast.metabolites.cys__L_c:,
        # H
        met.metabolites.gln__L_c: -0.12902,
        # M
        met.metabolites.met__L_c: -0.06451,
        # T
        met.metabolites.thr__L_c: -0.38706,
        # R
        met.metabolites.arg__L_c: -0.06451,
        # E
        met.metabolites.glu__L_c: -0.6451,
        # I
        met.metabolites.ile__L_c: -0.38706,
        # F
        met.metabolites.phe__L_c: -0.58059,
        # W
        met.metabolites.trp__L_c: -0.12902,
        # N
        met.metabolites.asn__L_c: -0.25804,
        # Q
        met.metabolites.gln__L_c: -0.32255,
        # L
        met.metabolites.leu__L_c: -0.83863,
        # P
        met.metabolites.pro__L_c: -0.32255,
        # Y
        met.metabolites.tyr__L_c: -0.19353,
        # D
        met.metabolites.arg__L_c: -0.51608,
        # G
        met.metabolites.gly_c: -0.51608,
        # K
        met.metabolites.lys__L_c: -0.90314,
        # S
        met.metabolites.ser__L_c: -0.83863,
        # V
        met.metabolites.val__L_c: -0.77412,
        # ATP
        met.metabolites.atp_c: -40.22199,
        # H2O
        met.metabolites.h2o_c: -40.22199,
        # Leghemoglobin
        met.metabolites.Hemo_AA: 1.0,
        # ADP
        met.metabolites.adp_c: 40.22199,
        # Phosphate
        met.metabolites.pi_c: 40.22199
    }
)



Hemo_Biosynthesis.add_metabolites(
    {
        met.metabolites.Hemo_DNA: -2.8e-05,
        
        met.metabolites.Hemo_RNA: -0.0029,
        
        met.metabolites.Hemo_AA: -0.997,

        met.metabolites.hemo_c: 1.0,
    }
)

# put in into extracellular space

EX_hemo.add_metabolites(
    {
        met.metabolites.hemo_c: -1.0,
        
        met.metabolites.hemo_e: 1.0
    }
)


# take it out of extracelllular space

Take_Out_Hemo.add_metabolites(
        {
            met.metabolites.hemo_e : -1.0
        }
)





# add reactions to model

met.add_reactions([hemo_DNA_reaction,
                   hemo_RNA_reaction,
                   hemo_AA_reaction,
                   Hemo_Biosynthesis,
                   Take_Out_Hemo,
                   EX_hemo])

In [264]:
met.reactions.BIOMASS_Mcapsulatus.bounds =  0.001, 0.001

In [265]:
new_objective = met.problem.Objective(met.reactions.Hemo_Biosynthesis.flux_expression)
met.objective = new_objective
met.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,6.986E-05,0,0.00%
ch4_e,EX_ch4_e,2.659,1,100.00%
cl_e,EX_cl_e,0.0002144,0,0.00%
cu2_e,EX_cu2_e,1.511E-06,0,0.00%
fe2_e,EX_fe2_e,6.12E-06,0,0.00%
h_e,EX_h_e,0.008193,0,0.00%
k_e,EX_k_e,0.0001765,0,0.00%
mg2_e,EX_mg2_e,0.0001234,0,0.00%
na1_e,EX_na1_e,3.915E-05,0,0.00%
no3_e,EX_no3_e,0.008195,0,0.00%


In [261]:
cobra.summary.MetaboliteSummary(metabolite = met.metabolites.hemo_c, model = met)

Percent,Flux,Reaction,Definition
Percent,Flux,Reaction,Definition


In [139]:
cobra.summary.reaction_summary.ReactionSummary(reaction = met.reactions.Hemo_Biosynthesis, model = met)

KeyError: 'Hemo_Biosynthesis'